In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [14]:
# Tornado data
tornado_df = pd.read_csv("us_tornado_dataset_1950_2021.csv")

# HPI data
hpi_df = pd.read_csv("united_states_housing.csv")

# ZIP to state mapping (downloaded from GitHub, or use a local file)
zip_ref_df = pd.read_csv("zip_code_database.csv", usecols=["zipcode", "state_abbr"])
zip_ref_df.columns = ["state", "Five-Digit ZIP Code"]
zip_ref_df = zip_ref_df[zip_ref_df["Five-Digit ZIP Code"].str.isnumeric()]
zip_ref_df["Five-Digit ZIP Code"] = zip_ref_df["Five-Digit ZIP Code"].astype(int)

print(zip_ref_df.head())

  state  Five-Digit ZIP Code
0    AL                35004
1    AL                35005
2    AL                35006
3    AL                35007
4    AL                35010


In [15]:
# Merge HPI with ZIP-to-state mapping
hpi_with_state_df = pd.merge(hpi_df, zip_ref_df, on="Five-Digit ZIP Code", how="left")

# Drop rows with missing values
hpi_with_state_df = hpi_with_state_df.dropna(subset=["state", "HPI"])

# Aggregate: average HPI per state and year
hpi_state_year_df = hpi_with_state_df.groupby(["state", "Year"])["HPI"].mean().reset_index()

In [16]:
# Rename columns to match for merging
tornado_df = tornado_df.rename(columns={"yr": "Year", "st": "state"})

# Optional: summarize tornadoes per state/year
# e.g., aggregate total injuries, fatalities, avg magnitude, etc.
agg_tornado_df = tornado_df.groupby(["state", "Year"]).agg({
    "mag": "mean",
    "inj": "sum",
    "fat": "sum",
    "len": "mean",
    "wid": "mean"
}).reset_index()


In [ ]:
merged_df = pd.merge(agg_tornado_df, hpi_state_year_df, on=["state", "Year"], how="inner")

# Drop rows with missing values
merged_df = merged_df.dropna()

In [22]:
print(merged_df.head())
X = merged_df.drop(columns=["HPI", "state"])
y = merged_df["HPI"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  state  Year       mag  inj  fat       len         wid         HPI
0    AK  2004  0.000000    0    0  0.000000    0.000000  150.768000
1    AK  2005  0.000000    0    0  0.000000    0.000000  166.564000
2    AL  1989  1.043478  484   22  5.647826  123.869565  100.000000
3    AL  1990  0.473684   74    0  1.831579   28.210526   99.140000
4    AL  1991  0.500000   33    5  0.680000   56.900000  100.636667


In [26]:
#early stopping
train_pool = Pool(X_train, y_train)
val_pool = Pool(X_test, y_test)

cat_model = CatBoostRegressor(
     iterations=5000,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    early_stopping_rounds=100,  # Stop if no improvement on test set
    verbose=100,
    random_seed=42
)


cat_model.fit(train_pool, eval_set=val_pool, use_best_model=True)
y_pred = cat_model.predict(X_test)

print("CatBoost Regressor (no past history, real data):")
r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2:.3f}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.3f}')

relative_error = mae / np.mean(y_test) * 100
print(f'Relative Error: {relative_error:.2f}%')

0:	learn: 89.0580995	test: 96.2345764	best: 96.2345764 (0)	total: 4.24ms	remaining: 21.2s
100:	learn: 66.6895622	test: 79.3370020	best: 79.2720613 (95)	total: 48.5ms	remaining: 2.35s
200:	learn: 59.8823987	test: 79.5399085	best: 78.8558579 (148)	total: 93.2ms	remaining: 2.23s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 78.85585785
bestIteration = 148

Shrink model to first 149 iterations.
CatBoost Regressor (no past history, real data):
R² score: 0.340
Mean Absolute Error (MAE): 51.346
Relative Error: 26.12%


In [27]:
#baseline
y_mean = np.full_like(y_test, y_train.mean())
mae_baseline = mean_absolute_error(y_test, y_mean)
r2_baseline = r2_score(y_test, y_mean)
print(f"Baseline MAE: {mae_baseline:.2f}, Baseline R²: {r2_baseline:.2f}")

Baseline MAE: 61.71, Baseline R²: -0.00
